In [ ]:
# install Hugging Face Libraries
!pip install peft
!pip install transformers datasets accelerate evaluate bitsandbytes loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
from transformers import AutoTokenizer

model_id="facebook/bart-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import  AutoModelForSequenceClassification
import torch

model =  AutoModelForSequenceClassification.from_pretrained(model_id, return_dict=True, num_labels=2)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print((model))

BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_la

In [ ]:
from peft import LoraConfig

# Define LoRA Config
lora_config = LoraConfig(
 r=6,
 lora_alpha=16,
 target_modules=["q_proj", "v_proj"],
 lora_dropout=0.05,
 bias="none",
 task_type= "SEQ_CLS"
)


In [ ]:
# prepare int-8 model for training
from peft import prepare_model_for_kbit_training
from peft import get_peft_model
model = prepare_model_for_kbit_training(model)



In [ ]:
# add LoRA adaptor
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 331,776 || all params: 140,344,322 || trainable%: 0.2364


In [ ]:
print(peft_model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BartForSequenceClassification(
      (model): BartModel(
        (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
          (layers): ModuleList(
            (0-5): 6 x BartEncoderLayer(
              (self_attn): BartSdpaAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=6, bias=False)
                  )
                  (lora_B

In [ ]:

import pandas as pd
import json


def jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            entry = json.loads(line.strip())
            data.append(entry)

    df = pd.DataFrame(data)
    return df

file_path = '/content/drive/MyDrive/sampled_20000.csv'
df = pd.read_csv(file_path)

df.head()

,Unnamed: 0,text,labels
0,0,"Hoo boy. A better question might be, how did w...",0
1,1,"There are several formats that could be used,...",0
2,2,Warblington railway station serves the Warblin...,0
3,3,Modern quantum machine learning (QML) method...,0
4,4,The way most people describe audio quality is ...,0


In [ ]:
file_path = '/content/drive/MyDrive/subtaskA_dev_monolingual.jsonl'
validation_df = jsonl_to_dataframe(file_path)

validation_df.head()

,text,label,model,source,id
0,Giving gifts should always be enjoyable. Howe...,1,bloomz,wikihow,0
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1,bloomz,wikihow,1
2,If you'd rather not annoy others by being rude...,1,bloomz,wikihow,2
3,If you're interested in visiting gravesite(s) ...,1,bloomz,wikihow,3
4,The following are some tips for becoming succe...,1,bloomz,wikihow,4


In [ ]:
%%capture
!pip install transformers  datasets

In [ ]:
from datasets import *
from transformers import RobertaModel, RobertaTokenizer

import torch
import torch.nn.functional as F

In [ ]:
df

,Unnamed: 0,text,labels
0,0,"Hoo boy. A better question might be, how did w...",0
1,1,"There are several formats that could be used,...",0
2,2,Warblington railway station serves the Warblin...,0
3,3,Modern quantum machine learning (QML) method...,0
4,4,The way most people describe audio quality is ...,0
...,...,...,...
39995,39995,\n\nThe royal family is famously known for its...,1
39996,39996,Qianjiang Motorcycle (also known as QJ Motor) ...,1
39997,39997,\n\nWe consider the boundary value problems fo...,1
39998,39998,1520.com is a domain name that was registered ...,1


In [ ]:
df['label']=df['labels'].astype('category')
df['target']=df['label'].cat.codes

df.head()

,Unnamed: 0,text,labels,label,target
0,0,"Hoo boy. A better question might be, how did w...",0,0,0
1,1,"There are several formats that could be used,...",0,0,0
2,2,Warblington railway station serves the Warblin...,0,0,0
3,3,Modern quantum machine learning (QML) method...,0,0,0
4,4,The way most people describe audio quality is ...,0,0,0


In [ ]:
df_val =validation_df

In [ ]:
df_val['label']=df_val['label'].astype('category')
df_val['target']=df_val['label'].cat.codes

df_val.head()

,text,label,model,source,id,target
0,Giving gifts should always be enjoyable. Howe...,1,bloomz,wikihow,0,1
1,Yveltal (Japanese: ユベルタル) is one of the main a...,1,bloomz,wikihow,1,1
2,If you'd rather not annoy others by being rude...,1,bloomz,wikihow,2,1
3,If you're interested in visiting gravesite(s) ...,1,bloomz,wikihow,3,1
4,The following are some tips for becoming succe...,1,bloomz,wikihow,4,1


In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
category_map = {code: category for code, category in enumerate(df['label'].cat.categories)}
category_map

{0: 0, 1: 1}

In [ ]:
df_test = pd.read_json(r"/content/drive/MyDrive/filtered_output_balanced.jsonl", lines=True)
df_test['label']=df_test['label'].astype('category')
df_test['target']=df_test['label'].cat.codes

df_test.head()

,text,label,model,source,id,target
0,A power of attorney is a valuable legal docume...,1,chatGPT,wikihow,1583,1
1,"Midget car racing, also known as midget auto ...",1,davinci,wikipedia,11554,1
2,\n\nStrategies for the Design of a Slide-o-Cam...,1,cohere,arxiv,26444,1
3,\n\nHow to Watch TV Without Cable\n\n Watching...,1,cohere,wikihow,3090,1
4,Fun Kids is a national children's and pop digi...,0,human,wikipedia,83755,0


In [ ]:
train_end_point = int(df.shape[0])
print(train_end_point)


df_train = df

print(df_train.shape, df_test.shape, df_val.shape)

40000
(40000, 5) (5003, 6) (5000, 6)


In [ ]:
from datasets import *
from datasets import Dataset, DatasetDict
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train.drop('label',axis=1))
# dataset_val = Dataset.from_pandas(df_val.drop('label',axis=1))
dataset_test = Dataset.from_pandas(df_test.drop('label',axis=1))
validation_dataset = Dataset.from_pandas(df_val.drop('label',axis=1))
dataset_train_shuffled = dataset_train.shuffle(seed=42)
dataset = DatasetDict({
    'train': dataset_train_shuffled,
     'val': validation_dataset,
    'test': dataset_test
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'labels', 'target'],
        num_rows: 40000
    })
    val: Dataset({
        features: ['text', 'model', 'source', 'id', 'target'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'model', 'source', 'id', 'target'],
        num_rows: 5003
    })
})

In [ ]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)
val_dataset = dataset['val'].shuffle(seed=42)

In [ ]:
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}
)

In [ ]:
ds = dataset

In [ ]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'text', 'labels', 'target'],
    num_rows: 40000
})

In [ ]:
max_input_length = 1024
def tokenize_text(batch):
  return tokenizer(batch["text"],
                   padding=True,
                   truncation=True,
                   return_token_type_ids=True,
                   max_length=max_input_length)

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_text, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_text, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5003 [00:00<?, ? examples/s]

In [ ]:
tokenized_val_dataset = validation_dataset.map(tokenize_text, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
columns=["target", "input_ids", "attention_mask"]

tokenized_train_dataset.set_format("torch", columns=columns)
tokenized_test_dataset.set_format("torch", columns=columns)
tokenized_val_dataset.set_format("torch", columns=columns)

In [ ]:
print(tokenized_val_dataset)

Dataset({
    features: ['text', 'model', 'source', 'id', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})


In [ ]:
BATCH_SIZE = 8

In [ ]:
class MyDataset(Dataset):
  def __init__(self, dataset, partition_key):
    self.dataset = dataset

  def __getitem__(self, index):
    return self.dataset[index]

  def __len__(self):
    return self.dataset.num_rows

In [ ]:
train_data = MyDataset(tokenized_train_dataset, partition_key="train")
test_data = MyDataset(tokenized_test_dataset, partition_key="test")

In [ ]:
val_data = MyDataset(tokenized_val_dataset, partition_key="val")

In [ ]:
validation_data = val_data

In [ ]:
from torch.utils.data import DataLoader, Dataset
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
validation_loader  = DataLoader(dataset=validation_data, batch_size=BATCH_SIZE)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model

BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=6, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=6, out_features=768, bias=False)
              )
              (lora_embedding_A): Para

In [ ]:
model.to(device)

BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=6, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=6, out_features=768, bias=False)
              )
              (lora_embedding_A): Para

In [ ]:
model.to(device)

BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=6, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=6, out_features=768, bias=False)
              )
              (lora_embedding_A): Para

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
base_param_count = count_parameters(peft_model)
print(base_param_count)

331776


In [ ]:
import time
lr = 5e-5
EPOCHS = 2

In [ ]:
def get_accuracy(y_pred, targets):
  predictions = torch.log_softmax(y_pred, dim=1).argmax(dim=1)
  accuracy = (predictions == targets).sum() / len(targets)
  return accuracy

# prompt: give a function to give f-score  like def get_accuracy(y_pred, targets):
#   predictions = torch.log_softmax(y_pred, dim=1).argmax(dim=1)
#   accuracy = (predictions == targets).sum() / len(targets)
#   return accuracy

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
def get_f1_score(y_pred, targets, average='macro'):
    predictions = torch.log_softmax(y_pred, dim=1).argmax(dim=1)
    f1 = f1_score(targets.cpu(), predictions.cpu(), average=average)
    return f1


In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
def train(model, train_loader, epochs, optimizer, loss_function, device):
    total_time = 0

    for epoch in range(epochs):
        interval = len(train_loader) // 5 if len(train_loader) >= 5 else 1
        total_train_loss = 0
        total_train_acc = 0

        start = time.time()
        model.train()

        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["target"].to(device)  # Make sure your dataset uses "labels"

            # BartForSequenceClassification returns loss and logits when labels are provided
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            logits = outputs.logits

            acc = get_accuracy(logits, labels)
            f_measure = get_f1_score(logits, labels)

            total_train_loss += loss.item()
            total_train_acc += acc.item()

            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % interval == 0:
                print("Batch: %s/%s | Training loss: %.4f | Accuracy: %.4f | F1: %.4f" %
                      (batch_idx + 1, len(train_loader), loss.item(), acc.item(), f_measure))

        train_loss = total_train_loss / len(train_loader)
        train_acc = total_train_acc / len(train_loader)

        end = time.time()
        hours, remainder = divmod(end - start, 3600)
        minutes, seconds = divmod(remainder, 60)

        print(f"\nEpoch {epoch+1} | Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")
        print("Time: {:0>2}:{:0>2}:{:05.2f}\n".format(int(hours), int(minutes), seconds))

        total_time += (end - start)

    avg_time = total_time / epochs
    hours, remainder = divmod(avg_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    print("Average time per epoch: {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train(peft_model, train_loader, EPOCHS, optimizer, loss_function, device)

Batch: 1000/5000 | Training loss: 0.0207 | Accuracy: 1.0000 | F1: 1.0000
Batch: 2000/5000 | Training loss: 0.0062 | Accuracy: 1.0000 | F1: 1.0000
Batch: 3000/5000 | Training loss: 0.0076 | Accuracy: 1.0000 | F1: 1.0000
Batch: 4000/5000 | Training loss: 0.0030 | Accuracy: 1.0000 | F1: 1.0000
Batch: 5000/5000 | Training loss: 0.0036 | Accuracy: 1.0000 | F1: 1.0000

Epoch 1 | Loss: 0.0977 | Accuracy: 0.9596
Time: 00:26:08.65

Batch: 1000/5000 | Training loss: 0.0016 | Accuracy: 1.0000 | F1: 1.0000
Batch: 2000/5000 | Training loss: 0.0050 | Accuracy: 1.0000 | F1: 1.0000
Batch: 3000/5000 | Training loss: 0.0221 | Accuracy: 1.0000 | F1: 1.0000
Batch: 4000/5000 | Training loss: 0.0008 | Accuracy: 1.0000 | F1: 1.0000
Batch: 5000/5000 | Training loss: 0.0006 | Accuracy: 1.0000 | F1: 1.0000

Epoch 2 | Loss: 0.0233 | Accuracy: 0.9923
Time: 00:26:08.64

Average time per epoch: 00:26:08.64


In [54]:
train(peft_model, train_loader, 2, optimizer, loss_function, device)

Batch: 1000/5000 | Training loss: 0.0032 | Accuracy: 1.0000 | F1: 1.0000
Batch: 2000/5000 | Training loss: 0.0010 | Accuracy: 1.0000 | F1: 1.0000
Batch: 3000/5000 | Training loss: 0.0702 | Accuracy: 1.0000 | F1: 1.0000
Batch: 4000/5000 | Training loss: 0.0011 | Accuracy: 1.0000 | F1: 1.0000
Batch: 5000/5000 | Training loss: 0.0131 | Accuracy: 1.0000 | F1: 1.0000

Epoch 1 | Loss: 0.0159 | Accuracy: 0.9947
Time: 00:26:07.71

Batch: 1000/5000 | Training loss: 0.0008 | Accuracy: 1.0000 | F1: 1.0000
Batch: 2000/5000 | Training loss: 0.0061 | Accuracy: 1.0000 | F1: 1.0000
Batch: 3000/5000 | Training loss: 0.0004 | Accuracy: 1.0000 | F1: 1.0000
Batch: 4000/5000 | Training loss: 0.0016 | Accuracy: 1.0000 | F1: 1.0000
Batch: 5000/5000 | Training loss: 0.0008 | Accuracy: 1.0000 | F1: 1.0000

Epoch 2 | Loss: 0.0113 | Accuracy: 0.9961
Time: 00:26:07.01

Average time per epoch: 00:26:07.36


In [ ]:

#train(peft_model, train_loader, EPOCHS, optimizer, loss_function, device)

In [ ]:
from sklearn.metrics import classification_report

def evaluate(model, test_loader, loss_function, device):
    interval = len(test_loader) // 5 if len(test_loader) >= 5 else 1

    total_test_loss = 0
    total_test_acc = 0

    all_preds = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["target"].to(device)  # true labels

            # Model outputs
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            acc = get_accuracy(logits, labels)
            f_measure = get_f1_score(logits, labels)

            total_test_loss += loss.item()
            total_test_acc += acc.item()

            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            if (batch_idx + 1) % interval == 0:
                print("Batch: %s/%s | Test loss: %.4f | Accuracy: %.4f | F1: %.4f" %
                      (batch_idx + 1, len(test_loader), loss.item(), acc.item(), f_measure))

    test_loss = total_test_loss / len(test_loader)
    test_acc = total_test_acc / len(test_loader)

    print(f"\nTest Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")

    # Print classification report
    print("\nClassification Report:\n")
    print(classification_report(all_labels, all_preds, digits=4))

In [ ]:
evaluate(peft_model, test_loader, loss_function, device) # 2 epochs


Batch: 125/626 | Test loss: 0.7867 | Accuracy: 0.7500 | F1: 0.6667
Batch: 250/626 | Test loss: 0.2084 | Accuracy: 0.8750 | F1: 0.8730
Batch: 375/626 | Test loss: 0.0007 | Accuracy: 1.0000 | F1: 1.0000
Batch: 500/626 | Test loss: 0.6046 | Accuracy: 0.8750 | F1: 0.8545
Batch: 625/626 | Test loss: 0.3950 | Accuracy: 0.8750 | F1: 0.8545

Test Loss: 0.5466 | Accuracy: 0.8548

Classification Report:

              precision    recall  f1-score   support

           0     0.9994    0.7097    0.8300      2501
           1     0.7750    0.9996    0.8731      2502

    accuracy                         0.8547      5003
   macro avg     0.8872    0.8547    0.8516      5003
weighted avg     0.8872    0.8547    0.8516      5003



In [ ]:
evaluate(peft_model, validation_loader, loss_function, device) # 2 epochs

Batch: 125/625 | Test loss: 0.3188 | Accuracy: 0.7500 | F1: 0.4286
Batch: 250/625 | Test loss: 0.0170 | Accuracy: 1.0000 | F1: 1.0000
Batch: 375/625 | Test loss: 2.6306 | Accuracy: 0.3750 | F1: 0.2727
Batch: 500/625 | Test loss: 0.0005 | Accuracy: 1.0000 | F1: 1.0000
Batch: 625/625 | Test loss: 1.6082 | Accuracy: 0.5000 | F1: 0.3333

Test Loss: 0.6554 | Accuracy: 0.8188

Classification Report:

              precision    recall  f1-score   support

           0     0.9182    0.7000    0.7944      2500
           1     0.7576    0.9376    0.8380      2500

    accuracy                         0.8188      5000
   macro avg     0.8379    0.8188    0.8162      5000
weighted avg     0.8379    0.8188    0.8162      5000



In [55]:
evaluate(peft_model, test_loader, loss_function, device) # 4 epochs


Batch: 125/626 | Test loss: 0.4754 | Accuracy: 0.8750 | F1: 0.8545
Batch: 250/626 | Test loss: 0.0131 | Accuracy: 1.0000 | F1: 1.0000
Batch: 375/626 | Test loss: 0.0002 | Accuracy: 1.0000 | F1: 1.0000
Batch: 500/626 | Test loss: 0.0082 | Accuracy: 1.0000 | F1: 1.0000
Batch: 625/626 | Test loss: 0.0003 | Accuracy: 1.0000 | F1: 1.0000

Test Loss: 0.1738 | Accuracy: 0.9479

Classification Report:

              precision    recall  f1-score   support

           0     0.9996    0.8960    0.9450      2501
           1     0.9058    0.9996    0.9504      2502

    accuracy                         0.9478      5003
   macro avg     0.9527    0.9478    0.9477      5003
weighted avg     0.9527    0.9478    0.9477      5003



In [56]:
evaluate(peft_model, validation_loader, loss_function, device) # 4 epochs

Batch: 125/625 | Test loss: 0.0008 | Accuracy: 1.0000 | F1: 1.0000
Batch: 250/625 | Test loss: 0.0022 | Accuracy: 1.0000 | F1: 1.0000
Batch: 375/625 | Test loss: 1.3547 | Accuracy: 0.6250 | F1: 0.3846
Batch: 500/625 | Test loss: 0.0002 | Accuracy: 1.0000 | F1: 1.0000
Batch: 625/625 | Test loss: 0.5432 | Accuracy: 0.7500 | F1: 0.4286

Test Loss: 0.3734 | Accuracy: 0.8802

Classification Report:

              precision    recall  f1-score   support

           0     0.8847    0.8744    0.8795      2500
           1     0.8758    0.8860    0.8809      2500

    accuracy                         0.8802      5000
   macro avg     0.8803    0.8802    0.8802      5000
weighted avg     0.8803    0.8802    0.8802      5000

